[View in Colaboratory](https://colab.research.google.com/github/todnewman/coe_training/blob/master/Basic_Reinforcement_Learning_Lunar_Lander.ipynb)

# Basic Reinforcement Learning

This example shows how a Lunar Lander can be trained to learn how to land between the flags on the ever-changing moon surface.

## Needed Libraries

These are libraries that will allow this code to run.  They are not installed in CoLaboratory by default so we need to load them.  They will load even through a firewall.

In [3]:
!pip install --upgrade keras
!pip install np_utils
!apt install swig
!apt install cmake
!pip install gym[all]
!pip install statistics


    100% |████████████████████████████████| 307kB 14.9MB/s 
Requirement not upgraded as not directly required: numpy>=1.9.1 in /usr/local/lib/python2.7/dist-packages (from keras) (1.14.5)
Requirement not upgraded as not directly required: scipy>=0.14 in /usr/local/lib/python2.7/dist-packages (from keras) (0.19.1)
    100% |████████████████████████████████| 51kB 24.6MB/s 
Requirement not upgraded as not directly required: h5py in /usr/local/lib/python2.7/dist-packages (from keras) (2.8.0)
Requirement not upgraded as not directly required: pyyaml in /usr/local/lib/python2.7/dist-packages (from keras) (3.13)
Requirement not upgraded as not directly required: six>=1.9.0 in /usr/local/lib/python2.7/dist-packages (from keras) (1.11.0)
  Found existing installation: Keras 2.1.6
    Uninstalling Keras-2.1.6:
      Successfully uninstalled Keras-2.1.6
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
 

7Progress: [ 72%] [##########################################................] 8Setting up swig (3.0.10-1.2) ...
7Progress: [ 81%] [###############################################...........] 87Progress: [ 90%] [####################################################......] 8
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cmake-data libarchive13 libjsoncpp1 liblzo2-2 librhash0 libuv1
Suggested packages:
  ninja-build lrzip
The following NEW packages will be installed:
  cmake cmake-data libarchive13 libjsoncpp1 liblzo2-2 librhash0 libuv1
0 upgraded, 7 newly installed, 0 to remove and 0 not upgraded.
Need to get 4,930 kB of archives.
After this operation, 25.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/main amd64 cmake-data all 3.9.1-1 [1,276 kB]
Get:2 http://archive.ubuntu.com/ubuntu artful/main amd64 liblzo2-2 amd64 2.08-1.2 [48.7 kB]


7Progress: [ 97%] [########################################################..] 8Processing triggers for libc-bin (2.26-0ubuntu2.1) ...

78Collecting gym[all]
    100% |████████████████████████████████| 1.5MB 10.9MB/s 
    100% |████████████████████████████████| 1.0MB 8.5MB/s 
    100% |████████████████████████████████| 430kB 11.2MB/s 
    100% |████████████████████████████████| 5.8MB 4.1MB/s 
    100% |████████████████████████████████| 3.3MB 5.4MB/s 
    100% |████████████████████████████████| 1.2MB 16.0MB/s 
    100% |████████████████████████████████| 768kB 18.8MB/s 
    100% |████████████████████████████████| 3.3MB 7.2MB/s 
    100% |████████████████████████████████| 409kB 20.9MB/s 
    100% |████████████████████████████████| 256kB 25.8MB/s 
  Running setup.py bdist_wheel for gym ... - \ |

 / done
  Stored in directory: /content/.cache/pip/wheels/cb/14/71/f4ab006b1e6ff75c2b54985c2f98d0644fffe9c1dddc670925
  Running setup.py bdist_wheel for Box2D-kengz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/75/ae/e5/8bc678d262caad94659c199c540550e59d03dd3bd3684d4f1a
  Running setup.py bdist_wheel for mujoco-py ... - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/91/0a/d1/1333dd27794b10433bb878356d2368de008f25f0df066d11d3
  Running setup.py bdist_wheel for PyOpenGL ... - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /content/.cache/pip/wheels/6c/00/7f/1dd736f380848720ad79a1a1de5272e0d3f79c15a42968fb58
  Running setup.py bdist_wheel for atari-py ... - \ |

 / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/ac/79/85/b21b404d3469c3028aea3b7a1dfe9d9bf9827a20cee6a118bd
  Running setup.py bdist_wheel for glfw ... - done
  Stored in directory: /content/.cache/pip/wheels/ca/aa/05/9b802212ed86ef800f54025059bac9e64f58dfaf17fb97e94e
  Running setup.py bdist_wheel for pycparser ... - \ done
  Stored in directory: /content/.cache/pip/wheels/c0/a1/27/5ba234bd77ea5a290cbf6d675259ec52293193467a12ef1f46
Successfully built gym Box2D-kengz mujoco-py PyOpenGL atari-py glfw pycparser
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade

## GenNet - Tod's RL Generic Neural Net class

This class is very useful and generalizes across a number of RL applications.  It works for other Atari games as well as my Raspberry Pi-based autonomous rover.  This support two neural nets, one fully connected network with two hidden layers that uses the ADAM optimizer and one Convolutional Neural network that has 2 hidden convolutional layers and one hidden fully connected layer (plus code commented out to add a third convolutional layer if desired).  The CNN uses the RMSProp Optimizer.

In [4]:
import numpy as np
import np_utils
import keras
#from keras.utils import np_utils
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

from keras.layers import Conv2D, Flatten, Dense, Activation, Dropout
#from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import Callback
from keras import optimizers
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.models import Sequential
import csv

import tensorflow as tf
tf.test.gpu_device_name()


class GenNet(object):

    def __init__(self, **kwargs):
        self.verbose = kwargs.get('verbose', False)
        self.nodes_1 = kwargs.get('nodes_1')
        self.nodes_2 = kwargs.get('nodes_2')
        self.nodes_3 = kwargs.get('nodes_3')
        self.num_actions = kwargs.get('num_actions')
        self.num_inputs = kwargs.get('num_inputs')

        self.load_weights = kwargs.get('load_weights', False)
        self.save_weights = kwargs.get('save_weights', False)

        self.log_file = kwargs.get('log_file')
        self.weights_file = kwargs.get('weights_file', None)
        self.obs_shape = kwargs.get('obs_shape')
        self.loss_log = []
        self.learning_rate = .00025

        self.model = self.nn()
        #self.model = self.cnn()

        if self.verbose:
            print(
                """
                    Creating neural net with options:
                    Nodes, layer 1: %d
                    Nodes, layer 2: %d
                    Load weights?: %s
                    Save weights?: %s
                    Weights file: %s
                    Log file: %s
                """
                % (self.nodes_1,
                   self.nodes_2,
                   str(self.load_weights),
                   str(self.save_weights),
                   self.weights_file,
                   self.log_file)
            )
            
    
    def cnn(self):
        input_shape = (1,84,110)
        model = Sequential()
        model.add(Conv2D(16, 8, strides=(4, 4),
                              padding='valid',
                              activation='relu',
                              input_shape=input_shape,
                              data_format='channels_first'
                              ))

        # Second convolutional layer
        model.add(Conv2D(32, 4, strides=(2, 2),
                              padding='valid',
                              activation='relu',
                              input_shape=input_shape,
                              data_format='channels_first'
                              ))
        '''
        # Third convolutional layer
        model.add(Conv2D(60, 3, strides=(1, 1),
                              padding='valid',
                              activation='relu',
                              input_shape=input_shape,
                              data_format='channels_first'
                              ))
        '''
        # Flatten the convolution output
        model.add(Flatten())

        # First dense layer
        model.add(Dense(256, activation='relu'
                        ))
        
        # Output layer
        model.add(Dense(self.num_actions))
        
        optimizer = optimizers.RMSprop(lr=self.learning_rate, rho=0.9, epsilon=1e-06)
        #optimizer = optimizers.Adam(lr=self.learning_rate)
        model.compile(loss='mean_squared_error',
                           optimizer=optimizer,
                           metrics=['accuracy'])
                           
        if self.load_weights and self.weights_file is not None:
            if self.verbose:
                print("Loading weights from %s" % self.weights_file)
            model.load_weights(self.weights_file)

        return model
        
        
    def nn(self):
        model = Sequential()

        # First layer.
        model.add(Dense(
            self.nodes_1, init='lecun_uniform',
            input_shape=(self.num_inputs,)
        ))
        #model.add(Activation('relu'))
        model.add(LeakyReLU(alpha=0.005))
        #model.add(Dropout(0.2))

        # Second layer.
        model.add(Dense(self.nodes_2, init='lecun_uniform'))
        #model.add(Activation('relu'))
        model.add(LeakyReLU(alpha=0.005))
        #model.add(Dropout(0.2))
        
        # Third layer.
        model.add(Dense(self.nodes_3, init='lecun_uniform'))
        #model.add(Activation('relu'))
        model.add(LeakyReLU(alpha=0.005))
        #model.add(Dropout(0.2))

        # Output layer.
        model.add(Dense(self.num_actions, init='lecun_uniform'))
        model.add(Activation('linear'))

        #optimizer = optimizers.RMSprop(lr=self.learning_rate, rho=0.9, epsilon=1e-06)
        optimizer = optimizers.Adam(lr=self.learning_rate)
        model.compile(loss="mse", optimizer=optimizer)
        

        if self.load_weights and self.weights_file is not None:
            if self.verbose:
                print("Loading weights from %s" % self.weights_file)
            model.load_weights(self.weights_file)

        return model

    def predict(self, state):
        #print ("****", self.model.predict(state.reshape(1, 1, 84, 110)))
        try:
            #return self.model.predict(state.reshape(1, 1, 84, 110),
            return self.model.predict(state.reshape(1, self.num_inputs),
                                      batch_size=1)
        except:
            raise Exception("PredictionError")

    def train(self, X, y, batch_size):
        history = LossHistory()
        self.model.fit(
            X, y, batch_size=batch_size,
            nb_epoch=1, verbose=0, callbacks=[history]
        )
        self.loss_log.append(history.losses)

    def log_results(self):
        # Save the results to a file so we can graph it later.
        with open(self.log_file, 'w') as lf:
            wr = csv.writer(lf)
            for loss_item in self.loss_log:
                wr.writerow(loss_item)
        self.loss_log = []

    def save_weights_file(self):
        if self.weights_file is not None:
            self.model.save_weights(self.weights_file, overwrite=True)


class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

Using TensorFlow backend.


## GenRL- Tod's Generic RL support Library

This class allows us to do basic RL stuff.  It does the interfacing with both the Environment simulation (OpenAI's GYM environment) as well as with the Neural Net action predictor.  It also handles data management and training.

In [0]:
import numpy as np
import random


class Gen_RL(object):

    def __init__(self, model, **kwargs):

        self.verbose = kwargs.get('verbose', False)
        self.num_actions = kwargs.get('num_actions')

        self.epsilon = kwargs.get('epsilon', 1)
        self.min_epsilon = kwargs.get('min_epsilon')
        self.batch_size = kwargs.get('batch_size')
        self.episodes = kwargs.get('episodes')
        self.frames = kwargs.get('frames', 0)
        self.replay_size = kwargs.get('replay_size')
        self.save_steps = kwargs.get('save_steps', 5000)
        self.gamma = kwargs.get('gamma', 0.95)
        self.num_inputs = kwargs.get('num_inputs')

        self.enable_training = kwargs.get('enable_training', True)

        self.replay = []
        self.steps = 0

        self.model = model
        self.counter = 0

        # If we send along frames, that replaces episodes as the number of
        # steps we're going to take. So we need to reduce epsilon by frame
        # in this case, not by "terminal" or episode.
        if self.frames != 0:
            self.has_terminal = False
            self.epsilon_divider = self.frames
        else:
            self.has_terminal = True
            self.epsilon_divider = self.episodes

        if self.verbose:
            print(
                """
                    Creating learner with options:
                    Starting epsilon: %d
                    Minimum epsilon: %f
                    Batch size: %d
                    Episodes: %d
                    Frames: %d
                    Replay (buffer) size: %d
                    Gamma: %f
                    Actions: %d
                    State inputs: %d
                """
                % (self.epsilon,
                   self.min_epsilon,
                   self.batch_size,
                   self.episodes,
                   self.frames,
                   self.replay_size,
                   self.gamma,
                   self.num_actions,
                   self.num_inputs)
            )

    def determine_action(self, state):
        # Choose an action.
        if random.random() < self.epsilon and self.enable_training is True:
            action = np.random.randint(0, self.num_actions)  # random
        else:
            # Get Q values for each action.
            qval = self.model.predict(state)
            action = (np.argmax(qval))

        return action

    def step(self, state, action, reward, new_state, terminal):
        # Experience replay storage.
        self.replay.append((state, action, reward, new_state))
        

        # If we've stored enough in our buffer, pop.
        if len(self.replay) > self.replay_size:
            self.replay.pop(0)

        # If we have enough to train, train.
        if len(self.replay) > self.batch_size:
            # Randomly sample our experience replay memory
            minibatch = random.sample(self.replay, self.batch_size)

            # Get training values.
            X_train, y_train = self.process_minibatch(minibatch, terminal)

            # Train the model on this batch.
            self.model.train(X_train, y_train, self.batch_size)
        
        # Decrement epsilon over time.
        if self.epsilon > self.min_epsilon:
            if (self.has_terminal and terminal) or self.has_terminal is False:
                self.epsilon -= (1.0 / (self.epsilon_divider))
        
                self.epsilon = max (0.05, self.epsilon)
           
        # Save weights?
        if self.model.save_weights and self.model.weights_file is not None:
            if self.steps % self.save_steps == 0 and self.steps > 0:
                print("Saving weights.")
                self.model.save_weights_file()

        self.steps += 1
        


    def process_minibatch(self, minibatch, terminal=False):
        X_train = []
        y_train = []
        # Loop through our batch and create arrays for X and y
        # so that we can fit our model at every step.
        for memory in minibatch:
            # Get stored values.
            old_state_m, action_m, reward_m, new_state_m = memory
            # Get prediction on old state.
            old_qval = self.model.predict(old_state_m)
            # Get prediction on new state.
            newQ = self.model.predict(new_state_m)
            # Get our best move. I think?
            maxQ = np.max(newQ)
            y = np.zeros((1, self.num_actions))
            y[:] = old_qval[:]
            # Check for terminal state.
            if not terminal:  # non-terminal state
                update = (reward_m + (self.gamma * maxQ))
            else:  # terminal state
                update = reward_m
            # Update the value for the action we took.
            y[0][action_m] = update
            #X_train.append(old_state_m.reshape(1, 84, 110))
            X_train.append(old_state_m.reshape(self.num_inputs,))
            y_train.append(y.reshape(self.num_actions,))

        X_train = np.array(X_train)
        y_train = np.array(y_train)

        return X_train, y_train


## Main Function

This section will set up the Neural Net to predict what actions the Lunar Lander should take.  It also governs the initialization of the environment simulation and allows the Lunar Lander to receive environmental state values from the environment simulation and then receive rewards based on the appropriateness of the action.

This will train to be able to perform this activity after about 5,000 episodes.

To Train the Lander, uncomment the line "train_or_show = train" and ensure that "show" is commented out.  Once a trained model has been saved, switch to "show" mode to load the model into the empty lunar lander and start it predicting actions.

In [1]:
"""
Implements Gen_RL in attempt to solve Gym's Lunar Lander environment.
This is the main function that calls the classes in later workbook blocks.
Better to execute this block last...
"""

import gym
from statistics import mean, stdev
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display


episodes =  8000
inputs = 8
actions = 4


# Just change these.
train_or_show = 'train'
#train_or_show = 'show'
weights_file = 'lunarlander'
log_file = 'loss_log.txt'

# Depends on the above.
if train_or_show == 'train':
    enable_training = True
    load_weights = False
    save_weights = True
else:
    enable_training = False
    load_weights = True
    save_weights = False

# Create the environment. You can change this to other Gym environments
# to experiment.
env = gym.make('LunarLander-v2')
print("action space: {0!r}".format(env.action_space))
print("observation space: {0!r}".format(env.observation_space))

obs_shape = env.observation_space.shape

# Set up the network.

network = GenNet(nodes_1=165, nodes_2=140, nodes_3=10, num_actions=actions, 
                   num_inputs=inputs, verbose=True,
                   load_weights=load_weights, weights_file=weights_file, 
                   obs_shape=obs_shape, save_weights=save_weights, 
                   log_file=log_file)
                  
                 

# Setup Gen RL's deep RL model.
rl = Gen_RL(network, episodes=episodes, num_actions=actions,
                  batch_size=32, min_epsilon=0.1, num_inputs=inputs,
                  replay_size=10000, gamma=0.95, verbose=True,
                  enable_training=enable_training)



# Keep track of things below.
rewards = []
results = []
max_steps = 600
repeat_action = 3

def show_state(env, step=0, info=""):
    plt.figure(3)
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("%s | Step: %d %s" % (env._spec.id,step, info))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())


# Run.
for i in range(episodes):
    # Get initial state.
    state = env.reset()

    terminal = False
    e_rewards = 0
    steps = 0

    while not terminal:
        steps += 1

        # Render the environment.
        #env.monitor.start('/tmp/cartpole-experiment-1', force=True)
        #env.render()

        # Send the state to Gen_RL, get our predicted action.
        action = rl.determine_action(state)

        # Repeat the same action if we want.
        for x in range(repeat_action):
            new_state, reward, terminal, _ = env.step(action)
        #show_state(env, steps)

        # Add the info to our experience replay for training.
        if enable_training:
            rl.step(state, action, reward, new_state, terminal)
            
        

        # Accumulate rewards.
        e_rewards += reward
        
        # If we died...
        if terminal:
            # If we aren't training, be more verbose.
            if not enable_training:
                if reward == -100:
                    result = 'Crashed'
                else:
                    result = 'Landed'
                print("%s! Score: %d" % (result, e_rewards))

            # Give us some info.
            rewards.append(e_rewards)
            if len(rewards) > 10:
                rewards.pop(0)
            e_rewards = 0

        # For experience replay.
        state = new_state

        # This helps us out if we get stuck in a crack.
        if steps > max_steps:
            print("Too many steps.")
            break

    # Every 10th episode, print out useful info.
    if i % 10 == 0 and i > 0:
        print("-"*80)
        print("Epsilon: %.5f" % rl.epsilon)
        print("Episode: %d" % i)
        print("Mean: %.2f\tMax: %d\tStdev: %.2f" %
              (mean(rewards), max(rewards), stdev(rewards)))
        results.append(mean(rewards))

# Just print out all of our mean rewards so we can plot them or
# do other fun things.
env.close()
for r in results:
    print(r)
network.log_results()

ImportError: ignored

## Google Drive authentication Stuff

I don't like this much, but right now this is how I can allow the RL Models to be saved to the Google Drive.

In [6]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 21515 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .


Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8.1) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting up python-apt (1.4.0~beta3build2) ...
Setting up python3-dbus (1.2.4-1bu

## More authentication code

Here's where we do the work to create a OCAML Fuse drive.  The next block is where we mkdir a drive and tag it as an ocamlfuse drive type.  That's where we'll find the data.

In [9]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive


In [11]:
!ls drive/



2007 FFB Class Cheatsheet.ods	Colab Notebooks
2007 FFB Keeper Cheatsheet.ods	Fusion Tables
2008 FB Keeper Cheatsheet.ods	Imported from Google Notebook - My Notebook.odt
baseball			Lessons from the Life of David.odt
Border Team.odt			narnia_lessons.odt
Colab				wrting


In [0]:
!cp /content/lunarlander drive/.